In [1]:
import sys; sys.path.insert(0,'..')
from geotaste import *

In [2]:
df=get_combined_df()

In [16]:
def draw_choropleth(
        df, 
        heatmap=True, 
        key='arrond_id', 
        value='count', 
        count_df=None, 
        fill_color="YlGn",
        fill_opacity=0.7,
        line_opacity=0.2,
        **kwargs):
    hmap=draw_heatmap(df,**kwargs) if heatmap else draw_map(df,**kwargs)
    cdf=get_arrond_counts(df) if count_df is None else count_df
    cmap = folium.Choropleth(
        get_geojson_arrondissement(),
        data=cdf,
        columns=[key, value],
        key_on="feature.id",
        fill_color=fill_color,
        fill_opacity=fill_opacity,
        line_opacity=line_opacity,
    )
    
    cdfi=cdf.set_index(key)
    vkeys=set()
    for s in cmap.geojson.data['features']:
        for k,v in dict(cdfi.loc[s['id']]).items():
            vkeys.add(k)
            s['properties'][k] = round(float(v),2)
        
    # and finally adding a tooltip/hover to the choropleth's geojson
    folium.GeoJsonTooltip([key]+list(vkeys)).add_to(cmap.geojson)
    cmap.add_to(hmap)

    return hmap


In [17]:
opts=dict(zoom_start=13, zoom=False, heatmap=True)

df1=get_mapdf(year=1920)
df2=get_mapdf(year=1930)

# 1920s
m1=draw_choropleth(df1,**opts)
m1

In [18]:
# 1930s
m2=draw_choropleth(df2, **opts)
m2

In [19]:
# compare_maps(m1,m2)

In [20]:
def get_iframe(map, width=600, height=400, return_str=False, **attrs):
    src=map.get_root().render().replace('"', '&quot;')
    
    styled_default=dict(
        # display='inline-block',
        # width='49%',
        margin='0 auto',
        border='0',
    )
    
    styled = {**styled_default, **attrs}
    stylestr = '; '.join(f'{k}:{v}' for k,v in styled.items())
    ostr=f'<iframe srcdoc="{src}" width="{width}" height="{height}" style="{stylestr}"></iframe>'
    return ostr if return_str else HTML(ostr)

In [21]:
def compare_maps(map1,map2,width=600,height=600,return_str=False,**kwargs):
    iframe1 = get_iframe(map1,width=width,height=height,return_str=True,float='left',**kwargs)
    iframe2 = get_iframe(map2,width=width,height=height,return_str=True,float='right',**kwargs)
    ostr=iframe1 + iframe2
    return ostr if return_str else HTML(ostr)

In [22]:
def compare_choropleths(df1,df2,return_str=False,**kwargs):
    opts={**dict(zoom_start=12, zoom=False, heatmap=True, value='perc'), **kwargs}
    
    m1=draw_choropleth(df1, **opts)
    m2=draw_choropleth(df2, **opts)
    
    ## diff
    cdf1=get_arrond_counts(df1).set_index('arrond_id')
    cdf2=get_arrond_counts(df2).set_index('arrond_id')
    diff_df = (cdf2-cdf1)
    
    odf=pd.DataFrame()
    for c in cdf1: 
        odf[c+'1']=cdf1[c]
        odf[c+'2']=cdf2[c]
        odf[c+'2-1']=diff_df[c]
    
    diff_opts = {**opts}
    diff_opts['heatmap']=False
    diff_opts['fill_color']='RdBu'
    m3=draw_choropleth(df1, count_df=diff_df.reset_index(), **diff_opts)
    
    htmlstr = compare_maps(m1,m2,return_str=True, height=400, width=600)
    htmlstr+= f'<center>{get_iframe(m3,return_str=True, height=600, width=800)}</center>'
    display(HTML(htmlstr))
    display(odf)
    return htmlstr if return_str else HTML(htmlstr)
    

In [38]:
df1=df[df.book_id.str.contains('joyce-')]
df2=df[df.book_id.str.contains('maugham-')]
df1.shape, df2.shape

((274, 75), (204, 75))

In [33]:
compare_choropleths(df1,df2)

,count1,count2,count2-1,perc1,perc2,perc2-1
arrond_id,,,,,,
1,1,2,1,0.409836,1.063830,0.653994
2,2,0,-2,0.819672,0.000000,-0.819672
3,0,0,0,0.000000,0.000000,0.000000
4,1,2,1,0.409836,1.063830,0.653994
5,33,19,-14,13.524590,10.106383,-3.418207
6,50,38,-12,20.491803,20.212766,-0.279037
7,52,35,-17,21.311475,18.617021,-2.694454
8,14,15,1,5.737705,7.978723,2.241018
9,4,6,2,1.639344,3.191489,1.552145


In [131]:
# compare_maps(m1,m2)

In [29]:
df1[df1.arrond_id==16]

,member_id,book_id,event_type,start_date,end_date,member_uris,member_names,member_sort_names,subscription_price_paid,subscription_deposit,...,borrow_count,purchase_count,circulation_years,updated_book,arrond_id,start_year,start_dec,first_coordinates,lat,lon
event_id,,,,,,,,,,,,,,,,,,,,,
